In [17]:
!pip install ftfy
import warnings
warnings.filterwarnings("ignore")
import ftfy
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re

from math import exp
from numpy import sign

from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

    100% |████████████████████████████████| 51kB 2.5MB/s 


Using TensorFlow backend.


In [12]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: cuntwars_user_id=vFHd55haQ6; _ga=GA1.3.1069537971.1546461315; __utma=68291539.1069537971.1546461315.1547285225.1547285225.1; __utmz=68291539.1547285225.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-datasets/2477/4140/training.1600000.processed.noemoticon.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1548743353&Signature=Gezf46EGvdr6PHE%2BXZ90qoEKEwn9RK%2Fl5aOSN4yFYmtkqhDpPA7%2FsYkMHr%2FIOoM7EzHSvFnb%2FqpgJbxQYuBrIF80Ft8RrZz8oy4oV5bBk%2FeKivtVOwfCL8dcgmX3bPxJJtG3OwX3Yec9CMMY1PVsw6qWnISKdcX4PMHOmbpzD%2FwM7fuyn22%2FQLlOg%2F7TwYrAvi2QAXp%2Bz1sNwpnbzv4HWytO7JBBdyjf820UmOkpuLAX%2FzvqlmxI3bd38fTkVoJwOCLqj4ZETDH9IoSP0FR2JdX%2FuBMsqHMC9zsFamee9ryF2zySBMnnxtkokXGFM%2B0kHe7xykgvwLdemeHvb5qmcg%3D%3D" -O "training.1600000.processed.noemoticon.csv.zip" -c

--2019-01-26 06:43:05--  https://storage.googleapis.com/kaggle-datasets/2477/4140/training.1600000.processed.noemoticon.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1548743353&Signature=Gezf46EGvdr6PHE%2BXZ90qoEKEwn9RK%2Fl5aOSN4yFYmtkqhDpPA7%2FsYkMHr%2FIOoM7EzHSvFnb%2FqpgJbxQYuBrIF80Ft8RrZz8oy4oV5bBk%2FeKivtVOwfCL8dcgmX3bPxJJtG3OwX3Yec9CMMY1PVsw6qWnISKdcX4PMHOmbpzD%2FwM7fuyn22%2FQLlOg%2F7TwYrAvi2QAXp%2Bz1sNwpnbzv4HWytO7JBBdyjf820UmOkpuLAX%2FzvqlmxI3bd38fTkVoJwOCLqj4ZETDH9IoSP0FR2JdX%2FuBMsqHMC9zsFamee9ryF2zySBMnnxtkokXGFM%2B0kHe7xykgvwLdemeHvb5qmcg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.70.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.70.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88031309 (84M) [application/octet-stream]
Saving to: ‘training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  83.95M 

In [13]:
!unzip 'training.1600000.processed.noemoticon.csv.zip'

Archive:  training.1600000.processed.noemoticon.csv.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [0]:

data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1")

In [19]:
data.iloc[:,0].nunique()

2

In [26]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz


--2019-01-26 06:53:49--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.9.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  45.1MB/s    in 28s     

2019-01-26 06:54:18 (56.2 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [0]:
data.iloc[:,5] = data.iloc[:,5].apply(lambda x : re.sub("@[\w]+", "", x))
data.iloc[:,5] = data.iloc[:,5].apply(lambda x : re.sub("[^a-zA-Z#\s]","", x))

In [29]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his Facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball Managed to sa...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i he...
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,not the whole crew


In [0]:
# Expand Contraction
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [0]:
from tqdm import tqdm
def clean_tweets(tweets, data):
    cleaned_tweets = []
    labels_d = []
    counter = 0
    for tweet in tqdm(tweets):
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            
            
            #expand contraction
            tweet = expandContractions(tweet)

            #stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            #stemming words
            tweet = PorterStemmer().stem(tweet)
            
            labels_d.append(data.iloc[counter, 0])
            counter+=1
            
            cleaned_tweets.append(tweet)

    return cleaned_tweets, labels_d

In [32]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

x_data = [x for x in data.iloc[:,5]]
print(x_data[0])
X_d, labels_d = clean_tweets(x_data, data)
print(len(X_d))
print(len(labels_d))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 183/1599999 [00:00<14:34, 1829.65it/s]

is upset that he cant update his Facebook by texting it and might cry as a result  School today also Blah


100%|██████████| 1599999/1599999 [13:25<00:00, 1987.26it/s]

1574436
1574436


In [0]:
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_d)

In [0]:
sequences_d = tokenizer.texts_to_sequences(X_d)

In [35]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 481554 unique tokens


In [36]:
data_d = pad_sequences(sequences_d, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_d tensor:', data_d.shape)

Shape of data_d tensor: (1574436, 140)


In [0]:
nb_words = min(MAX_NB_WORDS, len(word_index))

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for (word, idx) in word_index.items():
    if word in word2vec.vocab and idx < MAX_NB_WORDS:
        embedding_matrix[idx] = word2vec.word_vec(word)

In [53]:
for i in tqdm(range(len(labels_d))):
  if(labels_d[i]==4):
    labels_d[i]=1
labels_d = np.array(labels_d)
perm_d = np.random.permutation(len(data_d))
idx_train_d = perm_d[:int(len(data_d)*(TRAIN_SPLIT))]
idx_test_d = perm_d[int(len(data_d)*(TRAIN_SPLIT)):int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT))]
idx_val_d = perm_d[int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT)):]

data_train = ((data_d[idx_train_d]))
labels_train = ((labels_d[idx_train_d]))
data_test = ((data_d[idx_test_d]))
labels_test = ((labels_d[idx_test_d]))
data_val = ((data_d[idx_val_d]))
labels_val = ((labels_d[idx_val_d]))

perm_train = np.random.permutation(len(data_train))
data_train = data_train[perm_train]
labels_train = labels_train[perm_train]
perm_test = np.random.permutation(len(data_test))
data_test = data_test[perm_test]
labels_test = labels_test[perm_test]
perm_val = np.random.permutation(len(data_val))
data_val = data_val[perm_val]
labels_val = labels_val[perm_val]

100%|██████████| 1574436/1574436 [00:01<00:00, 1484688.25it/s]


In [56]:
max(labels_test)

1

In [57]:
data_train.shape

(944661, 140)

In [58]:
labels_train.shape

(944661,)

In [0]:
labels_train = list(labels_train)

labels_train = np.array(labels_train)

In [0]:
model = Sequential()
# Embedded layer
model.add(Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], 
                            input_length=MAX_SEQUENCE_LENGTH, trainable=False))
# Convolutional Layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layer
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 140, 300)          6000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 140, 32)           28832     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 70, 32)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 70, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               399600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total para

In [0]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

In [0]:
model.load_weights('epoch3.h5')

early_stop = EarlyStopping(monitor='val_loss', patience=3)

hist = model.fit(data_train, labels_train, \
        validation_data=(data_val, labels_val), \
        epochs=5, batch_size=256, shuffle=True, \
        callbacks=[early_stop])

Train on 944661 samples, validate on 314888 samples
Epoch 1/5
944661/944661 [==============================] - 736s 779us/step - loss: 0.4756 - acc: 0.7723 - val_loss: 0.4687 - val_acc: 0.7771
Epoch 2/5
338688/944661 [=========>....................] - ETA: 7:03 - loss: 0.4706 - acc: 0.7761

In [0]:
model.save_weights('epoch8.h5')